# Core 9 — Explain-only LangChain (core9_06)

본 노트북에서 LangChain은 **어떠한 판정, 예약, 점수 계산도 수행하지 않는다.**

- 입력: 이미 결정된 fallback / reservation 로그
- 출력: 사람이 읽을 **고정 포맷 설명문**
- 실패 시에도 파이프라인은 계속 진행된다.

설명 생성은:
- 재현 가능해야 하며
- 언제든 재실행 가능하고
- 핵심 산출물(Core 8/9 판단)과 분리된다.

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

# 입력 선택 (둘 중 하나만 존재해도 OK)
RESERVATION_PATH = Path("../artifact/core9/core9_04_reservation_log.csv")
FALLBACK_PATH    = Path("../artifact/core8/core8_06_fallback_decisions.csv")

EXPORT_DIR = Path("../artifact/core9")
EXPORT_DIR.mkdir(exist_ok=True)

OUT_PATH = EXPORT_DIR / "core9_06_explanations.csv"

In [2]:
if RESERVATION_PATH.exists():
    src = "core9_reservation"
    df = pd.read_csv(RESERVATION_PATH)
elif FALLBACK_PATH.exists():
    src = "core8_fallback"
    df = pd.read_csv(FALLBACK_PATH)
else:
    raise FileNotFoundError("No input log found for explanation.")

print("source:", src)
df.head()

source: core9_reservation


,run_id,case_id,antibody_id,step,risk_score_soms,risk_score_osc,risk_score_conflict,risk_score_total,forecast_hazard_level,forecast_rule_id,...,first_event_step,lead_time,reservation_evaluated,reservation_status,reservation_reason_code,reservation_horizon,reservation_rule_id,fallback_stage,fallback_reason_code,fallback_score
0,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,0,0.000,0.0,NaN,NaN,NONE,core9_03_v1,...,NaN,NaN,True,HOLD,REASON_MIN_STEPS_NOT_REACHED,5,core9_04_v1,HOLD,REASON_MIN_STEPS_NOT_REACHED,0.000
1,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,1,0.000,0.0,NaN,NaN,NONE,core9_03_v1,...,NaN,NaN,True,HOLD,REASON_MIN_STEPS_NOT_REACHED,5,core9_04_v1,HOLD,REASON_MIN_STEPS_NOT_REACHED,0.000
2,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,2,0.000,0.0,NaN,NaN,NONE,core9_03_v1,...,NaN,NaN,True,HOLD,REASON_MIN_STEPS_NOT_REACHED,5,core9_04_v1,HOLD,REASON_MIN_STEPS_NOT_REACHED,0.000
3,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,3,0.000,0.0,NaN,NaN,NONE,core9_03_v1,...,NaN,NaN,True,HOLD,REASON_MIN_STEPS_NOT_REACHED,5,core9_04_v1,HOLD,REASON_MIN_STEPS_NOT_REACHED,0.000
4,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,4,0.006,0.0,NaN,NaN,NONE,core9_03_v1,...,NaN,NaN,True,HOLD,REASON_MIN_STEPS_NOT_REACHED,5,core9_04_v1,HOLD,REASON_MIN_STEPS_NOT_REACHED,0.006


In [3]:
def normalize_columns(df, src):
    out = df.copy()

    if src == "core9_reservation":
        out["stage"]  = out["reservation_status"]
        out["reason"] = out["reservation_reason_code"]
        out["score"]  = out["risk_score_total"]

    else:  # core8 fallback
        out["stage"]  = out["fallback_stage"]
        out["reason"] = out["fallback_reason_code"]
        out["score"]  = out.get("fallback_score", np.nan)

    # 안전 캐스팅
    out["score"] = pd.to_numeric(out["score"], errors="coerce")

    return out

work = normalize_columns(df, src)
work[["stage", "reason", "score"]].head()

,stage,reason,score
0,HOLD,REASON_MIN_STEPS_NOT_REACHED,NaN
1,HOLD,REASON_MIN_STEPS_NOT_REACHED,NaN
2,HOLD,REASON_MIN_STEPS_NOT_REACHED,NaN
3,HOLD,REASON_MIN_STEPS_NOT_REACHED,NaN
4,HOLD,REASON_MIN_STEPS_NOT_REACHED,NaN


In [4]:
USE_LLM = True  # False로 두면 설명 생성 스킵

if USE_LLM:
    try:
        from langchain_openai import ChatOpenAI
        from langchain_core.prompts import ChatPromptTemplate

        llm = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0
        )

        prompt = ChatPromptTemplate.from_messages([
            ("system", "You explain governance decisions. Do not change them."),
            ("user", 
             "stage={stage}\n"
             "reason={reason}\n"
             "score={score}\n"
             "한 문장 한국어 설명.")
        ])

        LLM_READY = True
    except Exception as e:
        print("LLM init failed:", e)
        LLM_READY = False
else:
    LLM_READY = False  # LangChain 초기화 (실패 허용)

In [5]:
def explain_row(r):
    if not LLM_READY:
        return "[EXPLAIN_SKIPPED]"

    try:
        msg = prompt.format_messages(
            stage=r["stage"],
            reason=r["reason"],
            score=round(r["score"], 3) if pd.notna(r["score"]) else "NA"
        )
        return llm.invoke(msg).content

    except Exception as e:
        return f"[EXPLAIN_FAILED] {type(e).__name__}" # 설명 생성 함수
    
work["explain"] = work.apply(explain_row, axis=1)
work[["stage", "reason", "score", "explain"]].head(10)

,stage,reason,score,explain
0,HOLD,REASON_MIN_STEPS_NOT_REACHED,NaN,"단계가 보류 중이며, 최소 단계에 도달하지 못했기 때문에 이유가 있습니다."
1,HOLD,REASON_MIN_STEPS_NOT_REACHED,NaN,"단계가 보류 중이며, 최소 단계에 도달하지 못했기 때문에 이유가 있습니다."
2,HOLD,REASON_MIN_STEPS_NOT_REACHED,NaN,"단계가 보류 중이며, 최소 단계 수에 도달하지 못했기 때문에 결정이 내려지지 않았습니다."
3,HOLD,REASON_MIN_STEPS_NOT_REACHED,NaN,"단계가 보류 중이며, 최소 단계 수에 도달하지 못했기 때문에 결정이 내려지지 않았습니다."
4,HOLD,REASON_MIN_STEPS_NOT_REACHED,NaN,"단계가 보류 중이며, 최소 단계에 도달하지 못했기 때문에 이유가 있습니다."
5,HOLD,REASON_MIN_STEPS_NOT_REACHED,NaN,"단계가 보류 중이며, 최소 단계 수에 도달하지 못했기 때문에 이유가 있습니다."
6,HOLD,REASON_MIN_STEPS_NOT_REACHED,NaN,"단계가 보류 중이며, 최소 단계에 도달하지 못했기 때문에 이유가 있습니다."
7,HOLD,REASON_MIN_STEPS_NOT_REACHED,NaN,"단계가 보류 중이며, 최소 단계에 도달하지 못했기 때문에 이유가 있습니다."
8,HOLD,REASON_MIN_STEPS_NOT_REACHED,NaN,"단계가 보류 중이며, 최소 단계 수에 도달하지 못했기 때문에 결정이 내려지지 않았습니다."
9,HOLD,REASON_MIN_STEPS_NOT_REACHED,0.02725,이 결정은 최소 단계에 도달하지 못했기 때문에 보류 상태로 유지됩니다.


In [6]:
export_cols = [
    c for c in work.columns
    if c not in ["__index_level_0__"]
]

work[export_cols].to_csv(OUT_PATH, index=False)
print("exported:", OUT_PATH)

work["explain"].value_counts(dropna=False) # 운영 안전성 체크

exported: ../artifact/core9/core9_06_explanations.csv


explain
단계가 보류 중이며, 최소 단계 수에 도달하지 못했기 때문에 결정이 내려지지 않았습니다.    20
단계가 보류 중이며, 최소 단계에 도달하지 못했기 때문에 이유가 있습니다.            12
단계가 보류 중이며, 최소 단계 수에 도달하지 못했기 때문에 이유가 있습니다.           9
이 결정은 최소 단계에 도달하지 못했기 때문에 보류 상태로 유지됩니다.               6
단계가 보류 중이며, 최소 단계에 도달하지 못했기 때문에 결정이 내려지지 않았습니다.       6
                                                     ..
이 결정은 기준 내에 있으며, 점수는 0.121로 평가되었습니다.                  1
이 결정은 기준 내에 있으며, 점수는 0.117로 평가되었습니다.                  1
이 결정은 기준 내에 있으며, 점수는 0.113으로 평가되었습니다.                 1
이 결정은 기준 내에서 이루어졌으며, 점수는 0.112입니다.                    1
이 결정은 기준 내에 있으며, 점수는 0.219로 평가되었습니다.                  1
Name: count, Length: 115, dtype: int64